# Condensates for Trappist 1-e models from Bower et al. (2024)

In [ ]:
import copy
import logging

import numpy as np
import pandas as pd
from pathlib import Path

from atmodeller import debug_logger
from atmodeller.constraints import ElementMassConstraint, SystemConstraints, BufferedFugacityConstraint, TotalPressureConstraint
from atmodeller.thermodata.redox_buffers import IronWustiteBuffer
from atmodeller.core import GasSpecies, Species, LiquidSpecies, SolidSpecies
from atmodeller.interior_atmosphere import Planet, InteriorAtmosphereSystem
from atmodeller.initial_solution import InitialSolutionRegressor, InitialSolutionSwitchRegressor, InitialSolutionDict, InitialSolutionLast
from atmodeller.utilities import earth_oceans_to_kg, bulk_silicate_earth_abundances
from atmodeller.output import Output

logger = debug_logger()
logger.setLevel(logging.INFO)

Parameters for the simulations.

In [ ]:
number_of_realisations = 5000
training_steps = 200

In [ ]:
surface_temperature = 1800

Set the equilibrium temperature of the planet, which is the temperature we will cool the atmosphere to.

In [ ]:
equilibrium_temperature = 280

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. 2021; Mantle mass determined assuming same proportion as Earth

In [ ]:
mantle_mass = 2.912E24
planet_mass = mantle_mass / (1-0.295334691460966)
trappist1e = Planet(surface_temperature=equilibrium_temperature, planet_mass=planet_mass, surface_radius=5.861E6)

Species to consider, including condensed C and H2O

In [ ]:
# Only CHON are uncommented for initial testing

H2O_g = GasSpecies("H2O")
H2_g = GasSpecies("H2")
O2_g = GasSpecies("O2")
CO_g = GasSpecies("CO")
CO2_g = GasSpecies("CO2")
CH4_g = GasSpecies("CH4")
N2_g = GasSpecies("N2")
NH3_g = GasSpecies("NH3")
#S2_g = GasSpecies("S2")
#H2S_g = GasSpecies("H2S")
#SO2_g = GasSpecies("SO2")
#SO_g = GasSpecies("SO")
#Cl2_g = GasSpecies("Cl2")
H2O_l = LiquidSpecies("H2O")
C_cr = SolidSpecies("C")

# All species
#species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, S2_g, H2S_g, SO2_g, SO_g, Cl2_g, H2O_l, C_cr])

# CHON only
species = Species([H2O_g, H2_g, O2_g, CO_g, CO2_g, CH4_g, N2_g, NH3_g, C_cr, H2O_l])

system = InteriorAtmosphereSystem(species=species, planet=trappist1e)

Get element abundances for constraints

In [ ]:
# Below allows us to use the training output for testing purposes
# trappist1e_with_sols_path = trappist1e_with_sols_training_path  
trappist1e_with_sols_path = Path(f"trappist1e_{surface_temperature}K_with_sols_{number_of_realisations}its")
output = Output.read_pickle(trappist1e_with_sols_path.with_suffix('.pkl'))
with_solubility_data = output(to_dataframes=True)

In [ ]:
# Create a dictionary with the series. Only the CHON constraints are uncommented
data = {
    'H_total': with_solubility_data["H_total"]["atmosphere_mass"],
    # 'S_total': with_solubility_data["S_total"]["atmosphere_mass"],
    'N_total': with_solubility_data["N_total"]["atmosphere_mass"],
    'O_total': with_solubility_data["O_total"]["atmosphere_mass"],
    'C_total': with_solubility_data["C_total"]["atmosphere_mass"],
    "H_moles": with_solubility_data["H_total"]["atmosphere_moles"],
    "C_moles": with_solubility_data["C_total"]["atmosphere_moles"],
    "O_moles": with_solubility_data["O_total"]["atmosphere_moles"],
    # "O2_g": with_solubility_data["O2_g"]["fugacity"],
    # 'Cl_total': with_solubility_data["Cl_total"]["atmosphere_mass"]
}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data)

# This was to test reordering the data to facilitate using InitialSolutionLast
# df["C/O"] = df['C_moles']/df["O_moles"]
# df["O/C"] = 1/df["C/O"]
# df["H/O"] = df["H_moles"]/df["O_moles"]

# Testing seems to show that ordering by total O is most helpful for InitialSolutionLast to find
# a solution
df= df.sort_values(by=["O_total", "C/O"])
df

In [ ]:
# Note that we must update the minimum log10 pressure
initial_solution = InitialSolutionLast(species=species, min_log10_pressure=-75)

In [ ]:
trappist1e_cooled_path = Path(f"trappist1e_{equilibrium_temperature}K_{number_of_realisations}its")

In [ ]:
# Regressor performance is seemingly worse than using ordered data and the previous solution
# trappist1e_initial_solution = Path(f"trappist1e_280K_200its_fulltoll.pkl")
# initial_solution = InitialSolutionRegressor.from_pickle(trappist1e_initial_solution.with_suffix(".pkl"), species=species, fit=False, partial_fit=False, partial_fit_batch_size=50, min_log10_pressure=-75)

In [ ]:
for row in df.itertuples(index=True):
    # Save the index to allow us to correlate the condensed atmospheres with the high temperature
    # origin.
    index = row.Index
    extra = {'index': index}
    constraints = SystemConstraints([
        ElementMassConstraint("H", row.H_total),
        # ElementMassConstraint("S", row.S_total),
        ElementMassConstraint("N", row.N_total),
        ElementMassConstraint("O", row.O_total),
        # TotalPressureConstraint(1),
        ElementMassConstraint("C", row.C_total),
        # ElementMassConstraint("Cl", row.Cl_total),
        ]
    )
    # lm solver "always" finds a solution, although sometimes the residual is high. By contrast
    # "hybr" with default parameters seems to always find the right solution, or fails.
    system.solve(constraints, factor=10, initial_solution=initial_solution, max_attempts=20, extra_output=extra, errors="ignore") # , method='lm')

In [ ]:
system.failed_solves

In [ ]:
system.output(file_prefix=trappist1e_cooled_path, to_excel=True, to_pickle=True)